# Leaderboards Analysis

Based on our current scouting, as well as The Blue Alliance's OPR score, and any other criteria we have, create a ranked list of teams within the competition.

*run all Data notebooks before any of the analysis notebooks.*

In [1]:
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_notebook, show

import glob
import logging

import numpy as np
import pandas as pd

import sys

output_notebook()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger('Leaderboard_Analysis')

# Climb point lookup
CLIMB_POINTS = dict([
    ('Traversal',15),
    ('Traverse',15), # app defaults to traverse
    ('High',10),
    ('Mid',6),
    ('Low',4),
    ('None',0)])

CLIMB_NAME = dict([
    (15,'Traversal'),
    (10,'High'),
    (6,'Mid'),
    (4,'Low'),
    (0,'None')])

FIRST_AVG = 4
LAST_AVG = 4
TOTAL_AVG = FIRST_AVG + LAST_AVG


Loading BokehJS ...

In [2]:
tba_matches = pd.read_csv('tba_matches.csv', index_col=0)
tba_oprs = pd.read_csv('tba_oprs.csv', index_col=0)
app_event_summary = pd.read_csv('app_event_summary.csv', index_col=0)
app_match_summary = pd.read_csv('app_match_summary.csv', index_col=0)

app_match_summary.sort_values(['team','match'])
# Let's look at the number of matches we have scouting data for.
# If there are more than 6, let's calculate a scouting score for the first 3 matches, and the most recent 3 matches.
# This will allow us to see if they had early issues that they resolved that might mean their OPR/Total event scouting score is not representative.
# Likewise if the most recent score was lower, then they may have some reliability issues with the robot.
# Also, determine the highest climb position.

# This feels clunky. Might be a more elegant way to do this.
# extract all six climb placements and save them without the alliance name
cl = tba_matches[['red1','red1_climb']].rename(columns={'red1':'team','red1_climb':'climb'})
cl=pd.concat([cl,tba_matches[['red2','red2_climb']].rename(columns={'red2':'team','red2_climb':'climb'})])
cl=pd.concat([cl,tba_matches[['red3','red3_climb']].rename(columns={'red3':'team','red3_climb':'climb'})])
cl=pd.concat([cl,tba_matches[['blue1','blue1_climb']].rename(columns={'blue1':'team','blue1_climb':'climb'})])
cl=pd.concat([cl,tba_matches[['blue2','blue2_climb']].rename(columns={'blue2':'team','blue2_climb':'climb'})])
cl=pd.concat([cl,tba_matches[['blue3','blue3_climb']].rename(columns={'blue3':'team','blue3_climb':'climb'})])

# use points to sort, and only grab the maximum value
cl['climb_points']=cl.apply( lambda row: CLIMB_POINTS[row.climb], axis=1)
tba_climb_summary=cl.groupby("team").agg(highest_climb_points = pd.NamedAgg(column = 'climb_points', aggfunc = 'max'),
                                        climb_success_pct = pd.NamedAgg(column = 'climb_points', aggfunc = lambda x: 100 * (sum(x.where(x==0,other=1))/x.size ) )
                                        )
tba_climb_summary['tba_highest_endgame_position']=tba_climb_summary.apply( lambda row: CLIMB_NAME[row.highest_climb_points], axis=1)




In [4]:
app_team_summary = app_match_summary.groupby('team').agg( matches = pd.NamedAgg(column='match', aggfunc='count'),
                                      highest_climb_points = pd.NamedAgg(column='hanger_points', aggfunc='max'),
                                      first_avg = pd.NamedAgg(column='scouting_points', aggfunc = lambda x:  sum(x.head())/FIRST_AVG if x.size >= TOTAL_AVG else 0),
                                      last_avg = pd.NamedAgg(column='scouting_points', aggfunc = lambda x: sum(x.tail())/LAST_AVG if x.size >=TOTAL_AVG else 0))

app_team_summary['highest_endgame_position']=app_team_summary.apply( lambda row: CLIMB_NAME[row.highest_climb_points], axis=1)



teams=tba_oprs.merge(app_event_summary,on='team',how="left")
teams=teams.merge(app_team_summary, on='team', how="left")
teams=teams.merge(tba_climb_summary, on='team', how="left")
teams=teams[['team','oprs','scouting_points',"matches","first_avg","last_avg","highest_endgame_position","tba_highest_endgame_position","climb_success_pct"]]
teams=teams.round()

In [5]:
source = ColumnDataSource(teams)

columns = [
        TableColumn(field="team", title="Team",width=50),
        TableColumn(field="matches", title="Matches", width=75),
        # Because The Blue Alliance Data is taken from the score, we trust it over the scouted position.
        #TableColumn(field="highest_endgame_position", title="Max Climb (Scout)"),
        TableColumn(field="tba_highest_endgame_position", title="Max Climb (TBA)"),
        TableColumn(field="climb_success_pct", title="Climb Success Pct", width=175),
        TableColumn(field="oprs", title="OPR from TBA", width=175),
        TableColumn(field="scouting_points", title="Average Scouting Points", width=200),
        TableColumn(field="first_avg", title="Avg Scouting from first", width=200),
        TableColumn(field="last_avg", title="Avg Scouting from last", width=200),
    ]
data_table = DataTable(source=source, columns=columns, width=1000,height=1200,
        sortable=True,
        editable=False,
        reorderable=True)


# Leaderboard

The table below shows their Offensive Power Rating Score from TBA, as well as the Average Scouting Score of the matches so far.
If our scouting is accurate, and the OPR is higher than our Average Scouting Score, then OPR might be higher than their overall value as an alliance partner.
If the OPR is lower than our average scouting score, then they may be a good alliance pick that may not be noticed by other teams.

We also display the first few and last few scouting averages to indicate if there are changes in match performance.


In [6]:
show(data_table)